# Fake News Classication

Read the two datasets:
1. ABC news article titles for 14 years
2. The Examiner news articles

## Data setup and cleaning

In [2]:
import pandas

data_set_loc = '/Users/koo01a/Desktop/fake_news'

# Read in white wine data 
# https://www.kaggle.com/therohk/million-headlines
abc = pandas.DataFrame(pandas.read_csv(data_set_loc+"/abcnews-date-text.csv", sep=','))

# Read in red wine data 
# https://www.kaggle.com/therohk/examine-the-examiner
examiner = pandas.DataFrame(pandas.read_csv(data_set_loc+"/examiner-date-text.csv", sep=','))

print(abc.info())
print(examiner.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103665 entries, 0 to 1103664
Data columns (total 2 columns):
publish_date     1103665 non-null int64
headline_text    1103665 non-null object
dtypes: int64(1), object(1)
memory usage: 16.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3089781 entries, 0 to 3089780
Data columns (total 2 columns):
publish_date     int64
headline_text    object
dtypes: int64(1), object(1)
memory usage: 47.1+ MB
None


In [3]:
print(abc.head())
from lolviz import *
g = listviz(['hi','mom',{3,4},{"parrt":"user"}])
g.render(view=True) # render graphviz.files.Source object

   publish_date                                      headline_text
0      20030219  aba decides against community broadcasting lic...
1      20030219     act fire witnesses must be aware of defamation
2      20030219     a g calls for infrastructure protection summit
3      20030219           air nz staff in aust strike for pay rise
4      20030219      air nz strike to affect australian travellers


'Source.gv.pdf'

Assign examiner as fake (fake=1) and abs as not fake (fake=0)

In [4]:
# Add `type` column to `red` with value 1
examiner['fake'] = 1

# Add `type` column to `white` with value 0
abc['fake'] = 0

# Append `white` to `red`
articles = pandas.concat([examiner,abc]).sort_index().reset_index(drop=True)[:100000]
#articles = examiner.append(abc, ignore_index=True)

In [5]:
print(articles.head())

   publish_date                                      headline_text  fake
0      20100101       100 Most Anticipated books releasing in 2010     1
1      20030219  aba decides against community broadcasting lic...     0
2      20100101       10 best films of 2009 - What's on your list?     1
3      20030219     act fire witnesses must be aware of defamation     0
4      20100101  10 days of free admission at Lan Su Chinese Ga...     1


In [6]:
articles['headline_text'] = [str(headline) for headline in articles['headline_text']]

In [10]:
from keras.preprocessing.text import *

AttributeError: module 'pandas.core.computation' has no attribute 'expressions'

In [7]:


MAX_WORDS=30000

tokenizer = Tokenizer()

tokenizer.fit_on_texts(articles['headline_text'])

X = tokenizer.texts_to_matrix(articles['headline_text'], mode='binary')
y = y=np.ravel(articles['fake'])

#tokenizer.texts_to_sequences(input)


/Users/koo01a/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


AttributeError: module 'pandas.core.computation' has no attribute 'expressions'

In [ ]:
len(tokenizer.word_index)
tokenizer.texts_to_matrix(["to in the", "the brisbane winter","surf"])
y

In [9]:
from keras.models import Sequential
from keras.layers import Dense

vocab_size = len(tokenizer.word_index) + 1

model = Sequential()

model.add(Dense(500, activation="relu", input_dim=len(tokenizer.word_index) + 1))
model.add(Dense(80, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=5, validation_split=.5, verbose=1)

AttributeError: module 'pandas.core.computation' has no attribute 'expressions'

In [8]:
model.save('fake-news-classifier.h5')

NameError: name 'model' is not defined

In [142]:
predictions = model.predict_classes(X)

100000/100000 [==============================] - 243s   


## Try some classification of arbitrary text 

In [144]:
input_text = "10 best holiday destinations."
input_vector = tokenizer.texts_to_matrix([input_text], mode='binary')
prob_fake = model.predict(input_vector)[0][0]

print("Classifier thinks this is%s fake (p=%s)." % (" not" if prob_fake < 0.5 else "", round(prob_fake,4)))

Classifier thinks this is fake (p=1.0).


## Incorrect classifications

In [145]:
articles['predictions'] = predictions
errors = [(articles['headline_text'][i], articles['fake'][i], articles['predictions'][i]) for i in range(0,len(articles)) if articles['fake'][i] != articles['predictions'][i]]


## Evaluate

In [146]:
score = model.evaluate(X, y,verbose=1)

print(score)

100000/100000 [==============================] - 223s   
[0.27820136805430268, 0.96328000000000003]


In [148]:
precision_score(y, predictions)

0.96487918439431641

In [149]:
recall_score(y, predictions)

0.96155999999999997

In [150]:
f1_score(y, predictions)

0.96321673278038211